In [1]:
import sys
import os
import numpy as np
import dill
import gzip
import logging


sys.path.append(os.path.abspath('../../surmise/emulationmethods'))
from AKSGP import Emulator

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        # logging.FileHandler('emulator_train.log', mode='w'),  # Log to file
        logging.StreamHandler()  # Log to console
    ]
)
logger = logging.getLogger(__name__)

# Load data

In [2]:
# Load training data
train_dir = 'simulation_data/Grad_Pb-Pb-2760GeV/train'
X = np.loadtxt(os.path.join(train_dir, 'X.txt'))
Ymean = np.loadtxt(os.path.join(train_dir, 'Ymean.txt'))
Ystd = np.loadtxt(os.path.join(train_dir, 'Ystd.txt'))

print(f"Arrays loaded from directory '{train_dir}'.")
print(f"Shapes of loaded arrays: {X.shape}, {Ymean.shape}, {Ystd.shape}")


# Load testing data
test_dir = 'simulation_data/Grad_Pb-Pb-2760GeV/test'

Xval = np.loadtxt(os.path.join(test_dir, 'X.txt'))
Ymeanval = np.loadtxt(os.path.join(test_dir, 'Ymean.txt'))
Ystdval = np.loadtxt(os.path.join(test_dir, 'Ystd.txt'))

print(f"Arrays loaded from directory '{test_dir}'.")
print(f"Shapes of loaded arrays: {Xval.shape}, {Ymeanval.shape}, {Ystdval.shape}")


Arrays loaded from directory 'simulation_data/Grad_Pb-Pb-2760GeV/train'.
Shapes of loaded arrays: (485, 17), (485, 110), (485, 110)
Arrays loaded from directory 'simulation_data/Grad_Pb-Pb-2760GeV/test'.
Shapes of loaded arrays: (93, 17), (93, 110), (93, 110)


# Train emulators

In [3]:
inpsize = 485; numobs= (0,4,7,34);
test = 22
emu = Emulator(X=X[:inpsize], Y_mean=Ymean[:inpsize, numobs], Y_std=Ystd[:inpsize, numobs])
emu.fit(kernel='AKS', nrestarts=5)

print("  Original means:\n", Ymean[:4, numobs],"\n")
print("  Original standard deviations:\n", Ystd[:4, numobs],"\n")

# Predict with standard deviation
means, std_devs = emu.predict(X[:4])
print("  Predicted means:\n", means,"\n")
print("  Predicted standard deviations:\n", std_devs,"\n")


2024-08-21 18:49:17,350 - AKSGP - INFO - Automatic kernel selection opted. Best kernel for each output dimension will be selected from the list of kernels:
   ['Matern12', 'Matern32', 'Matern52', 'RBF']

2024-08-21 18:49:17,353 - AKSGP - INFO - Shape of training arrays: (436, 17), (436, 4), (436, 4)
2024-08-21 18:49:17,353 - AKSGP - INFO - Shape of pseudo_test arrays: (49, 17), (49, 4), (49, 4)
2024-08-21 18:49:17,354 - AKSGP - INFO - Training GPs with all available kernels...
2024-08-21 18:49:17,354 - AKSGP - INFO -   Standardizing input space...
2024-08-21 18:49:17,357 - AKSGP - INFO -   Standardizing data...
2024-08-21 18:49:29,313 - AKSGP - INFO -   Trained GPs with Matern12 kernels.
2024-08-21 18:49:41,490 - AKSGP - INFO -   Trained GPs with Matern32 kernels.
2024-08-21 18:49:50,121 - AKSGP - INFO -   Trained GPs with Matern52 kernels.
2024-08-21 18:49:56,456 - AKSGP - INFO -   Trained GPs with RBF kernels.
2024-08-21 18:49:56,457 - AKSGP - INFO - Finding best kernels for each out

  Original means:
 [[1608.155095  402.4201     64.998956  370.298827]
 [1376.107143  398.155003   75.674266  371.891188]
 [1715.649143  495.888734   93.457043  462.45653 ]
 [1474.812429  439.662816   79.33832   407.198466]] 

  Original standard deviations:
 [[10.336379  3.906345  0.932647  3.59714 ]
 [ 7.11056   3.403601  1.004127  3.174749]
 [ 9.336446  4.044188  1.309451  3.773167]
 [ 8.152738  3.249399  1.106364  2.995906]] 

  Predicted means:
 [[1618.85299903  407.37921578   65.53142559  374.69780572]
 [1367.0657825   397.33566339   74.88825894  371.17218365]
 [1691.94349288  492.83212606   92.42994513  459.42087948]
 [1477.92119353  438.74938252   79.06886704  406.33153474]] 

  Predicted standard deviations:
 [[7.07801856 3.60085577 0.88891937 3.33164554]
 [6.02738479 3.23166481 0.94322045 3.01774484]
 [6.78157149 3.7374113  1.1735853  3.48718547]
 [6.50653146 3.08026423 1.0308838  2.8439806 ]] 

